# Notebook 05: Multi-Version Sequential Comparison (Layer 5)

This notebook demonstrates:
- Sequential pairwise comparison: V1->V2, V2->V3, V3->V4, ...
- Cumulative changelog and version timeline
- HTML report generation
- Change statistics and visualizations

In [ ]:
import sys
sys.path.insert(0, '..')

import yaml
import json
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from src.comparators.multi_version import MultiVersionComparator
from src.models.comparison_ir import MultiVersionReport

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('Multi-version comparator ready.')

## Load PDF Versions

Place your versioned PDFs in `notebooks/sample_pdfs/` and list them in order.

In [ ]:
# Update these paths to your versioned PDFs
PDF_VERSIONS = [
    'sample_pdfs/v1.pdf',
    'sample_pdfs/v2.pdf',
    'sample_pdfs/v3.pdf',
]

# Check which files exist
existing = [p for p in PDF_VERSIONS if Path(p).exists()]
missing = [p for p in PDF_VERSIONS if not Path(p).exists()]

print(f'Found {len(existing)} PDF versions: {existing}')
if missing:
    print(f'Missing: {missing}')
    print('Please add sample PDFs to run the full pipeline.')

In [ ]:
if len(existing) >= 2:
    comparator = MultiVersionComparator(
        pdf_paths=existing,
        config=config,
    )
    report = comparator.run()
    print(f'\n=== Multi-Version Report ===')
    print(f'Versions: {report.version_labels}')
    print(f'Total changes: {report.total_changes}')
    print(f'Changes by type: {report.changes_by_type}')
    print(f'Changes by pair: {report.changes_by_pair}')
else:
    print('Need at least 2 PDFs. Creating a synthetic report for demo...')
    report = None

## Visualize Change Statistics

In [ ]:
if report and report.pair_diffs:
    # Build DataFrame of changes per pair
    rows = []
    for diff in report.pair_diffs:
        counts = diff.change_counts
        rows.append({
            'pair': diff.version_label,
            **counts
        })
    df_stats = pd.DataFrame(rows)
    
    # Stacked bar chart
    change_cols = ['text', 'table', 'image', 'drawing']
    available_cols = [c for c in change_cols if c in df_stats.columns]
    
    if available_cols:
        fig = px.bar(
            df_stats, x='pair', y=available_cols,
            title='Changes per Version Pair by Type',
            labels={'value': 'Number of Changes', 'pair': 'Version Pair'},
            barmode='stack',
        )
        fig.show()
    
    print('\nDetailed statistics:')
    print(df_stats.to_string(index=False))
else:
    print('No report data available. Add sample PDFs to visualize.')

## Version Timeline

In [ ]:
if report and report.version_timeline:
    print(f'\n=== Element Version Timeline ({len(report.version_timeline)} elements tracked) ===')
    for entry in report.version_timeline[:20]:  # Show first 20
        print(f'\n  {entry.element_id} ({entry.element_type}):')
        for pair_label, description in entry.changes_by_pair.items():
            print(f'    {pair_label}: {description}')
else:
    print('No timeline data. Add sample PDFs and run comparison.')

## Export JSON Changelog

In [ ]:
if report:
    changelog = {
        'versions': report.version_labels,
        'total_changes': report.total_changes,
        'changes_by_type': report.changes_by_type,
        'pairwise_diffs': [
            {
                'pair': diff.version_label,
                'total_changes': diff.total_changes,
                'change_counts': diff.change_counts,
                'pages_added': diff.pages_added,
                'pages_removed': diff.pages_removed,
            }
            for diff in report.pair_diffs
        ],
    }
    
    output_path = '../output/changelog.json'
    Path('../output').mkdir(exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(changelog, f, indent=2)
    print(f'Changelog exported to: {output_path}')
    print(json.dumps(changelog, indent=2)[:1000])

In [ ]:
print('Done! Multi-version comparison pipeline validated.')